In [1]:
import os
# import Google Drive 套件
from google.colab import drive
# 將自己的雲端硬碟掛載上去
drive.mount('/content/gdrive')

os.chdir('./gdrive/MyDrive/Colab Notebooks/cheng_ta/hw1')      # 檔案目錄

Mounted at /content/gdrive


In [ ]:
os.chdir('./gdrive/MyDrive/Colab Notebooks/cheng_ta/hw1')      # 檔案目錄

In [ ]:
# import shutil
# copyPath = './gdrive/MyDrive/Colab Notebooks/cheng_ta/hw1/train_data'
# pastePath = './train_data'
# shutil.copytree(copyPath, pastePath)

'./train_data'

In [ ]:
!nvidia-smi

Mon Mar 22 06:21:28 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# import sys
# sys.path.append('./gdrive/MyDrive/Colab Notebooks/cheng_ta/hw1')

In [2]:
import importlib
import random
import pickle
import logging
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import os
import time
import fun_cls as fc
import DrBC_modle as Dm
import function as func
from sklearn.model_selection import train_test_split

# load train data

In [3]:
# load data in trainDatas list
trainPath = './train_data/10000/'
trainFile = os.listdir(trainPath)
trainDatas = []
# for i in trainFile:
#     data = trainPath + i
#     with open(data, 'rb') as f:
#         trainDatas.append(pickle.load(f))
for i in trainFile:
    trainDatas.append(trainPath + i)

In [5]:
len(trainDatas)

67

In [4]:
valid_size = 0.05
train, valid = train_test_split(trainDatas, test_size=valid_size)

# module define

In [8]:
epochs = 100
lr = 0.001

In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
DrBC_module = Dm.DrBC_module(Layer=5, device=device).to(device)
lossfun = Dm.lossf(device=device, testN=5000)
opt = torch.optim.Adam(DrBC_module.parameters(), lr=lr, betas=(0.9, 0.999), eps=1e-08, weight_decay=0)
device, len(list(DrBC_module.parameters()))

(device(type='cuda', index=0), 38)

In [12]:
logger = func.create_logger('./loggers/', '5layer_logger_10000-2.txt')
#logger = func.create_logger('./gdrive/MyDrive/Colab Notebooks/cheng_ta/hw1/loggers/', '5layer_logger_10000.txt')

In [11]:
# for i in logger.handlers:
#   logger.removeHandler(i)
# logger.handlers = []

In [13]:
train_loss = []
val_loss = []

In [14]:
DrBC_module.load_state_dict(torch.load('./modules/DrBC_5Layer_10000.pt'))
#DrBC_module.load_state_dict(torch.load('./gdrive/MyDrive/Colab Notebooks/cheng_ta/hw1/modules/5_layer_10000_point/DrBC_5Layer_10000.pt'))

<All keys matched successfully>

In [ ]:
module_save_path = './modules/DrBC_5Layer_10000.pt'
loss_save_path = './modules/5Layers_loss.pickle'
# module_save_path = './gdrive/MyDrive/Colab Notebooks/cheng_ta/hw1/modules/DrBC_5Layer_10000.pt'
# loss_save_path = './gdrive/MyDrive/Colab Notebooks/cheng_ta/hw1/modules/5Layers_loss.pickle'

start = time.time()
logger.info('start at {}'.format(start))

for epoch in range(1, epochs+1):
    logger.info('Epoch {}/{}'.format(epoch, epochs))
    logger.info('-' * 10)
    
    for phase in ['train', 'val']:
        if phase == 'train':
            DrBC_module.train(True)  # Set model to training mode
            running = train
        else:
            DrBC_module.train(False)  # Set model to evaluate mode
            running = valid
        ls = []
        
        for path in running:
            try:  
                with open(path, 'rb') as f:
                    G = pickle.load(f)
                output = DrBC_module(G)
                loss = lossfun(output, G.scoreList)
                if phase == 'train':
                    opt.zero_grad()
                    loss.backward()
                    opt.step()
                
                logger.info('{}:  {}/{}, loss:{}'.format(phase, epoch, epochs, loss.item()))
                ls.append(loss.item())

            except Exception as e:
                logger.error(e, exc_info=True)

        if phase == 'train':
            train_loss.append(ls)

        elif phase == 'val':
            val_loss.append(ls)
        loss_dict = {'train':train_loss, 'val':val_loss}
        with open(loss_save_path, 'wb') as f:
            pickle.dump(loss_dict, f)

    train_time = time.time() - start
    torch.save(DrBC_module.state_dict(), module_save_path)
    logger.info('train time {}, {}/{}'.format(train_time, epoch, epochs))

2021-03-22 08:11:41,053 INFO start at 1616400701.05367
2021-03-22 08:11:41,055 INFO Epoch 1/100
2021-03-22 08:11:41,056 INFO ----------
2021-03-22 08:11:48,058 INFO train:  1/100, loss:49.01279830932617
2021-03-22 08:11:53,546 INFO train:  1/100, loss:49.01285171508789
2021-03-22 08:11:59,182 INFO train:  1/100, loss:49.01284408569336
2021-03-22 08:12:04,944 INFO train:  1/100, loss:49.012840270996094
2021-03-22 08:12:10,501 INFO train:  1/100, loss:49.01285171508789
2021-03-22 08:12:16,306 INFO train:  1/100, loss:49.01286315917969
2021-03-22 08:12:29,675 INFO train:  1/100, loss:49.012874603271484
2021-03-22 08:12:35,632 INFO train:  1/100, loss:49.0128059387207
2021-03-22 08:12:44,430 INFO train:  1/100, loss:49.012874603271484
2021-03-22 08:12:55,176 INFO train:  1/100, loss:49.01285934448242
2021-03-22 08:13:07,554 INFO train:  1/100, loss:49.01285171508789
2021-03-22 08:13:20,315 INFO train:  1/100, loss:49.01283264160156
2021-03-22 08:13:34,158 INFO train:  1/100, loss:49.012844

In [ ]:
list(DrBC_module.parameters())[2].grad

tensor([[-3.1811e-05,  0.0000e+00,  3.7338e-03, -3.9361e-02, -4.1629e-05,
          1.0846e-04,  0.0000e+00,  5.2710e-04, -2.0568e-02,  0.0000e+00,
         -2.9760e-02,  1.7371e-03, -3.9132e-02,  2.9969e-03, -1.1920e-02,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          6.2036e-03, -8.7824e-03,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         -5.9737e-04, -2.1687e-02, -2.5961e-02,  0.0000e+00, -3.6056e-02,
          0.0000e+00, -2.9821e-02,  8.6391e-03, -4.5501e-02, -2.3277e-02,
          0.0000e+00, -5.5303e-03, -7.9124e-03, -1.8834e-02, -4.0921e-02,
          0.0000e+00,  0.0000e+00, -3.2854e-02, -1.9750e-03,  0.0000e+00,
          0.0000e+00, -8.5776e-03, -1.4956e-02,  0.0000e+00, -4.1537e-03,
          6.9897e-03,  0.0000e+00, -1.5777e-02,  0.0000e+00, -1.3589e-02,
          1.6265e-02, -2.6415e-02,  0.0000e+00,  0.0000e+00,  1.4470e-02,
          0.0000e+00,  1.4925e-03, -2.1319e-02,  0.0000e+00,  0.0000e+00,
          0.0000e+00, -1.4259e-02, -5.

In [ ]:
output = DrBC_module(train[5])

In [ ]:
loss = lossfun(output, train[5].scoreList)

In [ ]:
output, loss

(tensor([[ 3.0010],
         [ 4.3244],
         [ 4.1092],
         ...,
         [-0.0694],
         [-0.0687],
         [-0.0720]], device='cuda:0', grad_fn=<AddmmBackward>),
 tensor([[212.2686]], device='cuda:0', grad_fn=<DivBackward0>))

In [ ]:
loss.backward()

RuntimeError: ignored

In [ ]:
output[3000:3030]

tensor([[-0.0286],
        [ 0.0063],
        [-0.0685],
        [-0.0684],
        [-0.0690],
        [-0.0377],
        [-0.0382],
        [-0.0423],
        [-0.0420],
        [-0.0537],
        [-0.0516],
        [-0.0528],
        [-0.0525],
        [-0.0137],
        [-0.0692],
        [-0.0534],
        [-0.0542],
        [-0.0720],
        [-0.0115],
        [-0.0445],
        [-0.0412],
        [-0.0516],
        [-0.0378],
        [-0.0523],
        [-0.0246],
        [-0.0523],
        [-0.0408],
        [-0.0685],
        [-0.0547],
        [-0.0275]], device='cuda:0', grad_fn=<SliceBackward>)

In [ ]:
train[5].degreeCoeffient

{0: tensor([0.0066, 0.0115, 0.0113, 0.0156, 0.0136, 0.0124, 0.0209, 0.0197, 0.0194,
         0.0169, 0.0150, 0.0187, 0.0249, 0.0235, 0.0205, 0.0363, 0.0285, 0.0256,
         0.0229, 0.0242, 0.0265, 0.0363, 0.0324, 0.0285, 0.0265, 0.0342, 0.0324,
         0.0363, 0.0265, 0.0274, 0.0265, 0.0324, 0.0342, 0.0342, 0.0324, 0.0309,
         0.0324, 0.0363, 0.0285, 0.0388, 0.0285, 0.0296, 0.0388, 0.0363, 0.0419,
         0.0235, 0.0342, 0.0256, 0.0324, 0.0285, 0.0363, 0.0324, 0.0388, 0.0459,
         0.0342, 0.0342, 0.0459, 0.0342, 0.0388, 0.0388, 0.0459, 0.0459, 0.0459,
         0.0324, 0.0363, 0.0419, 0.0419, 0.0388, 0.0459, 0.0342, 0.0419, 0.0419,
         0.0459, 0.0459, 0.0388, 0.0459, 0.0388, 0.0459, 0.0459, 0.0324, 0.0388,
         0.0459, 0.0419, 0.0459, 0.0459, 0.0419, 0.0459, 0.0459, 0.0419, 0.0459,
         0.0459, 0.0459, 0.0459, 0.0459]),
 1: tensor([0.0044, 0.0076, 0.0076, 0.0063, 0.0058, 0.0073, 0.0074, 0.0057, 0.0098,
         0.0054, 0.0094, 0.0121, 0.0116, 0.0119, 0.0103, 0.0

# try

In [ ]:
output = DrBC_module(trainDatas[0])
trues = trainDatas[0].scoreList
loss = lossfun(output, trues)

In [ ]:
loss

tensor([[214.4099]], device='cuda:0', grad_fn=<MulBackward0>)

In [ ]:
loss.backward()

In [ ]:
list(DrBC_module.parameters())[20].grad

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.4153,  4.5197,  0.0918,  ...,  0.0864,  0.3594,  4.6415],
        ...,
        [-0.5673, -6.7819, -0.1336,  ..., -0.1331, -0.5012, -6.9675],
        [-0.0663, -1.4174, -0.0211,  ..., -0.0273, -0.0702, -1.4608],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
       device='cuda:0')

In [ ]:
x = torch.tensor([5])
x.item()

5

In [ ]:
importlib.reload(Dm)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
DrBC_module = Dm.DrBC_module(Layer=5, device=device).to(device)

In [ ]:
DrBC_module.load_state_dict(torch.load('./modules/5_Layer_5000_point/DrBC_5Layer.pt'))

<All keys matched successfully>

In [ ]:
D = torch.load('./ff.pt')

In [ ]:
myG = fc.myGraph()

In [ ]:
myG.scoreList = D['betweenness_vec'].T.tolist()[0]

In [ ]:
adj = D['adjacency_matrix']
myG.nodeDict = {}
for i in range(adj.shape[0]):
    c = []
    for j in range(adj.shape[1]):
        if adj[i, j] == 1.:
            c.append(j)
    myG.nodeDict[i] = c

In [ ]:
myG.degreeCoeffient = D['neighbor_matrix']

In [ ]:
output = DrBC_module(myG)
output.shape

torch.Size([5000, 1])

In [ ]:
pred = output.cpu().detach()[:, 0]
truths = np.array(myG.scoreList)
accuracy = func.top_n_accuracy(pred, truths, n=250)
accuracy

0.0

In [ ]:
torch.sum(adj[0, :]), torch.sum(adj.T[0, :])

(tensor(138.), tensor(138.))

In [ ]:
import os

In [ ]:
os.getcwd()

'/content'

In [ ]:
os.listdir()

['.config', 'gdrive', 'sample_data']